In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_rows = 46000
import re
from currency_converter import CurrencyConverter
c = CurrencyConverter()

In [71]:
df = pd.read_csv('D:\\UW\\2nd semester\\Webscrapping\\Project\\cars_final2.csv', sep=',', low_memory = False)

In [72]:
df.shape

(46157, 22)

In [73]:
df.columns

Index(['Unnamed: 0', 'Body Type', 'Condition', 'Cylinders', 'Engine',
       'Exterior Color', 'For Sale By', 'Interior Color', 'Make', 'Mileage',
       'Model', 'Price', 'Transmission', 'Vehicle Title', 'Warranty', 'Year',
       'Number of Cylinders', 'Drive Type', 'Fuel Type', 'Drivetrain', 'Fuel',
       'Engine Description'],
      dtype='object')

In [74]:
df.loc[df['Body Type'].isnull(), 'Body Type'] = 'None'
df.loc[df['Body Type'].str.contains('(?i)sedan'), 'Body Type'] = 'Sedan'
df.loc[df['Body Type'].str.contains('(?i)coupe'),'Body Type'] = 'Coupe'
df.loc[df['Body Type'].str.contains('(?i)pick'),'Body Type'] = 'Pickup'
df.loc[df['Body Type'].str.contains('(?i)suv'),'Body Type'] = 'SUV'
df.loc[df['Body Type'].str.contains('(?i)wagon'),'Body Type'] = 'Wagon'
df.loc[df['Body Type'].str.contains('(?i)sport utility'),'Body Type'] = 'SUV'
df.loc[df['Body Type'].str.contains('(?i)van'),'Body Type'] = 'Van'
df.loc[df['Body Type'].str.contains('4'or'5'),'Body Type'] = 'Sedan'
df.loc[df['Body Type'].str.contains('2'or'3'),'Body Type'] = 'Coupe'
df.loc[df['Body Type'].str.contains('(?i)hatch'),'Body Type'] = 'None'
df.loc[df['Body Type'].str.contains('--'),'Body Type'] = 'None'
df.loc[df['Body Type'].str.contains('(?i)convertible'or'(?i)cabriolet'),'Body Type'] = 'Convertible '

for brand in df['Make'].unique():
    for model in df.loc[df['Make']==brand, 'Model'].unique():
        try:
            df.loc[(df['Make']==brand) & (df['Model']==model) & (df['Body Type']=='None'), 'Body Type'] = df.loc[(df['Make']==brand) & (df['Model']==model), 'Body Type'].value_counts()[0:1].index.tolist()
        except:
            pass

types = ['SUV', 'Sedan', 'Coupe', 'Not defined', 'Pickup','Convertible', 'Van', 'Wagon']
df.loc[df['Body Type'].str.contains('|'.join(types))==False,'Body Type'] = 'Other'

In [75]:
df['Body Type'].value_counts(dropna = False)

SUV             11890
Sedan           10954
Coupe            6788
Pickup           6603
Other            3758
Convertible      3108
Van              1756
Wagon            1300
Name: Body Type, dtype: int64

In [76]:
df.loc[df['Condition'].str.contains('(?i)new', na = False), 'Condition'] = 'New'
df.loc[df['Condition'].str.contains('(?i)used', na = False), 'Condition'] = 'Used'
df.loc[df['Condition'].str.contains('(?i)1', na = False), 'Condition'] = 'Used'

In [77]:
df['Condition'].value_counts(dropna = False)

Used                   35511
NaN                     4862
New                     3353
Certified pre-owned     2431
Name: Condition, dtype: int64

In [78]:
df.loc[df['Cylinders'].isnull(), 'Cylinders'] = df.loc[df['Cylinders'].isnull(), 'Number of Cylinders']
df.loc[df['Cylinders'].str.contains('(?i)v6', na=False), 'Cylinders']=6
df.loc[df['Cylinders'].str.contains('(?i)v8', na=False), 'Cylinders']=8
df.loc[df['Cylinders'].str.contains('(?i)v10', na=False), 'Cylinders']=10
df.loc[df['Cylinders'].str.contains('(?i)v12', na=False), 'Cylinders']=12
df.loc[df['Cylinders'].str.contains('(?i)3', na=False), 'Cylinders']=3
df.loc[df['Cylinders'].str.contains('(?i)4', na=False), 'Cylinders']=4
df.loc[df['Cylinders'].str.contains('(?i)6', na=False), 'Cylinders']=6
df.loc[df['Cylinders'].str.contains('(?i)8', na=False), 'Cylinders']=8
df.loc[df['Cylinders'].str.contains('(?i)10', na=False), 'Cylinders']=10
df.loc[df['Cylinders'].str.contains('(?i)12', na=False), 'Cylinders']=12
cyl = ['1','2','3','4','5','6','8','9','10','11','12']
df.loc[df['Cylinders'].str.contains('|'.join(cyl))==False,'Cylinders'] = None

for i in range(1,13):
    df.loc[df['Engine'].str.contains('(?i)i'+str(i), na=False), 'Cylinders'] = i
    df.loc[df['Engine'].str.contains('(?i)v'+str(i), na=False), 'Cylinders'] = i
    df.loc[df['Engine'].str.contains(str(i)+'(?i) cyl', na=False), 'Cylinders'] = i
    df.loc[df['Engine'].str.contains(str(i)+'(?i)-cyl', na=False), 'Cylinders'] = i
    
for brand in df['Make'].unique():
    for model in df.loc[df['Make']==brand, 'Model'].unique():
        try:
            df.loc[(df['Make']==brand) & (df['Model']==model) & (df['Cylinders'].isnull()), 'Cylinders'] = df.loc[(df['Make']==brand) & (df['Model']==model), 'Cylinders'].value_counts()[0:1].index.tolist()
        except:
            pass

df.loc[df['Cylinders'].isnull() == False, 'Cylinders'] = df.loc[df['Cylinders'].isnull()==False, 'Cylinders'].astype(str).astype(int)

In [79]:
df['Cylinders'].value_counts(dropna = False)

8.0     16042
4.0     14175
6.0     12872
NaN      1506
12.0      657
10.0      303
3.0       283
5.0       149
7.0       104
2.0        61
1.0         5
Name: Cylinders, dtype: int64

In [80]:
df.loc[df['Engine'].str.contains('(?i)cha', na = False), 'Turbo'] = 'Yes'
df.loc[df['Engine'].str.contains('(?i)tur', na = False), 'Turbo'] = 'Yes'
df.loc[df['Turbo']!='Yes', 'Turbo'] = 'No'
df['Turbo'].value_counts(dropna = False)

No     35964
Yes    10193
Name: Turbo, dtype: int64

In [81]:
df.loc[df['Engine'].isnull(), 'Engine'] = df.loc[df['Engine'].isnull(), 'Engine Description']
df.loc[df['Engine'].str.contains('(?i)electr', na=False), 'Cylinders']  = 0

df.loc[df['Engine'].isnull(), 'Engine'] = 'None'
for j in range (1,10):
    for k in range(0,10):
        df.loc[df['Engine'].str.contains(str(j)+'.'+str(k)+'(?i)L', na=False), 'Engine'] = str(j)+'.'+str(k) #5.0L
        df.loc[df['Engine'].str.contains(str(j)+'.'+str(k)+'(?i) L', na=False), 'Engine'] = str(j)+'.'+str(k) #5.0 L
        df.loc[df['Engine'].str.contains(str(j)+'.'+str(k)+'(?i) liter', na=False), 'Engine'] = str(j)+'.'+str(k) #5.0 Liter
        df.loc[df['Engine'].str.contains(str(j)+'.'+str(k), na=False), 'Engine'] = str(j)+'.'+str(k) #5.0
        df.loc[df['Engine'].str.contains(str(j)+'(?i)l', na=False), 'Engine'] = str(j)+'.'+str(k) #5L
        df.loc[df['Engine'].str.contains(str(j)+'(?i) l', na=False), 'Engine'] = str(j)+'.'+str(k) #5 L
        
for i in range (1000,10000,100):
    df.loc[df['Engine'].str.contains('(?i)'+str(i), na=False), 'Engine'] = str(i/1000)

for i in range (100,1000,10):
    df.loc[df['Engine'].str.contains('(?i)'+str(i), na=False), 'Engine'] = str(i/100)
    
df.loc[df['Engine'].str.contains('(?i)electr', na=False), 'Engine']  = 0
df.loc[df['Engine'].str.contains('(?i)--', na=False), 'Engine']  = 'None'

for i in range(1,13):
    df.loc[df['Engine'].str.contains('(?i)i'+str(i), na=False), 'Engine'] = 'None'
    df.loc[df['Engine'].str.contains('(?i)v'+str(i), na=False), 'Engine'] = 'None'
    df.loc[df['Engine'].str.contains(str(i)+'(?i) cyl', na=False), 'Engine'] = 'None'
    df.loc[df['Engine'].str.contains(str(i)+'(?i)-cyl', na=False), 'Engine'] = 'None'
    df.loc[df['Engine'] == str(i), 'Engine'] = 'None'

df.loc[df['Engine'].str.len()>3, 'Engine'] = 'None'
df.loc[df['Engine'].str.contains('\.', regex = True)==False, 'Engine'] = 'None'

for brand in df['Make'].unique():
    for model in df.loc[df['Make']==brand, 'Model'].unique():
        try:
            df.loc[(df['Make']==brand) & (df['Model']==model) & (df['Engine']=='None'), 'Engine'] = df.loc[(df['Make']==brand) & (df['Model']==model), 'Engine'].value_counts()[0:1].index.tolist()
        except:
            pass


In [82]:
df.loc[df['Engine']=='None', 'Engine'] = None
df.loc[df['Engine'].isnull() == False, 'Engine'] = df.loc[df['Engine'].isnull()==False, 'Engine'].astype(str).astype(float)

In [83]:
df['Engine'].value_counts(dropna = False)

2.0    6701
3.0    5501
NaN    4271
3.5    2530
4.0    1995
3.6    1833
1.2    1773
2.5    1499
1.0    1472
5.0    1256
1.4    1220
1.5    1213
5.7    1166
2.3    1117
6.7    1038
1.1     967
2.4     927
6.0     918
2.7     646
4.6     632
1.3     626
3.7     545
1.8     535
1.6     513
3.8     418
2.1     398
3.2     328
4.7     327
2.2     309
2.9     297
2.6     252
3.9     250
2.8     240
4.8     221
4.5     201
3.3     200
3.4     191
3.1     189
0.0     182
4.4     143
5.8     138
5.9     117
1.7     112
1.9     105
5.6      91
6.8      79
4.2      76
4.9      73
7.0      64
4.3      46
5.2      45
4.1      37
6.2      21
5.3      21
6.6      16
8.0      16
5.5      14
5.4       9
6.9       4
7.3       4
6.4       4
6.5       4
8.6       3
6.1       3
9.8       3
5.1       2
8.1       2
7.8       2
7.4       2
8.2       1
8.5       1
9.0       1
6.3       1
Name: Engine, dtype: int64

In [84]:
ex_colors = ['White','Black','Gray','Silver','Blue','Red','Green']
df.loc[df['Exterior Color']=='Grey', 'Exterior Color']='Gray'
for col in ex_colors:
    df.loc[df['Exterior Color'].str.contains('(?i)'+str(col)+'', na = False), 'Exterior Color'] = str(col)
df.loc[df['Exterior Color'].str.contains('|'.join(ex_colors))==False,'Exterior Color']='Other'
df.loc[df['Exterior Color'].isnull(),'Exterior Color']='Other'
df['Exterior Color'].value_counts(dropna = False)

White     9542
Black     9095
Other     8620
Gray      4824
Silver    4505
Red       4265
Blue      4259
Green     1047
Name: Exterior Color, dtype: int64

In [85]:
df.loc[df['For Sale By'].str.contains('(?i)own', na = False), 'For Sale By'] = 'Private Owner'
df.loc[df['For Sale By'].str.contains('(?i)private', na = False), 'For Sale By'] = 'Private Owner'
df.loc[df['For Sale By']!='Private Owner', 'For Sale By'] = 'Dealer'
df['For Sale By'].value_counts(dropna = False)

Dealer           42343
Private Owner     3814
Name: For Sale By, dtype: int64

In [86]:
in_colors = ['Black','Gray','Tan','Brown','Red']
df.loc[df['Interior Color']=='Grey', 'Interior Color']='Gray'
for col in in_colors:
    df.loc[df['Interior Color'].str.contains('(?i)'+str(col)+'', na = False), 'Interior Color'] = str(col)
df.loc[df['Interior Color'].str.contains('|'.join(in_colors))==False,'Interior Color']='Other'
df.loc[df['Interior Color'].isnull(),'Interior Color']='Other'
df['Interior Color'].value_counts(dropna = False)

Black    20270
Other    13700
Gray      6274
Tan       3394
Red       1280
Brown     1239
Name: Interior Color, dtype: int64

In [87]:
brands = ['Ford', 'Chevrolet', 'Mercedes-Benz', 'BMW', 'Toyota', 'Dodge', 'Ram', 'Audi', 'Porsche', 'Lexus', 'Cadillac', 'GMC']
for brand in brands:
    df.loc[df['Make'].str.contains('(?i)'+str(brand)+'', na = False), 'Make'] = str(brand)
df.loc[df['Make'].str.contains('|'.join(brands))==False,'Make']='Other'
df['Make'].value_counts(dropna = False)

Ford                       8037
Chevrolet                  6275
Mercedes-Benz              3678
BMW                        2282
Toyota                     1897
Dodge                      1641
Ram                        1337
Audi                       1203
Porsche                    1100
Lexus                      1097
Cadillac                   1039
GMC                        1004
Honda                       982
Volkswagen                  944
Land Rover                  855
Hyundai                     721
Mazda                       665
Infiniti                    651
Kia                         643
Jaguar                      630
Lincoln                     628
Buick                       626
Subaru                      579
Nissan                      570
Pontiac                     543
Jeep                        503
Acura                       444
Chrysler                    422
Ferrari                     386
Maserati                    332
Bentley                     331
Volvo   

In [88]:
df.loc[df['Mileage'].str.contains('[A-Za-z]', na=False), 'Mileage'] = None
df.loc[df['Mileage'].str.contains('[0-9]',na=False)==False, 'Mileage'] = None
df.loc[df['Mileage'].isnull() == False, 'Mileage'] = df['Mileage'].str.replace(',','')
df.loc[df['Mileage'].isnull() == False, 'Mileage'] = df.loc[df['Mileage'].isnull()==False, 'Mileage'].astype(str).astype(int)
df.loc[df['Mileage']>999999, 'Mileage'] = 999999
df.loc[df['Mileage'].isnull(), 'Mileage'] = df['Mileage'].mean()
df['Mileage'].mean()

50977.20557711606

In [89]:
df['USD'] = df.loc[df['Price'].str.contains('(?i)u', na = False), 'Price'].str.extract('(\d+)')
df['CAD'] = df.loc[df['Price'].str.contains('(?i)c', na = False), 'Price'].str.extract('(\d+)').astype(int)*round(c.convert(1, "CAD", 'USD'))
df.loc[df['Price'].str.contains('(?i)u', na = False), 'Price'] = df['USD']
df.loc[df['Price'].str.contains('(?i)c', na = False), 'Price'] = df['CAD']
df.loc[df['Price'].str.contains('[A-Za-z]', na=False), 'Price'] = None
df.loc[df['Price'].str.contains('[0-9]',na=False)==False, 'Price'] = None
df.loc[df['Price'].isnull() == False, 'Price'] = df.loc[df['Price'].isnull()==False, 'Price'].astype(str).astype(int)
for brand in df['Make'].unique():
    df.loc[(df['Make']==brand) & (df['Price'].isnull()), 'Price'] = round(df.loc[(df['Make']==brand) & (df['Price'].isnull())==False, 'Price'].mean())

In [90]:
df.loc[df['Transmission'].str.contains('(?i)man', na = False), 'Transmission'] = 'Manual'
df.loc[df['Transmission']!='Manual', 'Transmission'] = 'Automatic'
df['Transmission'].value_counts(dropna = False)

Automatic    40950
Manual        5207
Name: Transmission, dtype: int64

In [91]:
df.loc[df['Vehicle Title'] == 'Clear', 'Vehicle Title'] = 'Clean'
df.loc[df['Vehicle Title'] == '--', 'Vehicle Title'] = 'Clean'
df.loc[df['Vehicle Title'] != 'Clean', 'Vehicle Title'] = 'Other'
df['Vehicle Title'].value_counts(dropna = False)

Clean    43603
Other     2554
Name: Vehicle Title, dtype: int64

In [92]:
df.loc[df['Warranty'].str.contains('(?i)no', na = False), 'Warranty'] = 'No'
df.loc[df['Warranty']=='Unspecified', 'Warranty'] = 'No'
df.loc[df['Warranty'].isnull(), 'Warranty'] = 'No'
df.loc[df['Warranty']!='No', 'Warranty'] = 'Yes'
df['Warranty'].value_counts(dropna = False)

No     41843
Yes     4314
Name: Warranty, dtype: int64

In [93]:
df.loc[df['Year'].str.contains('[A-Za-z]', na=False), 'Year'] = None
df.loc[df['Year'].str.contains('[0-9]',na=False)==False, 'Year'] = None
df.loc[df['Year'].isnull() == False, 'Year'] = df.loc[df['Year'].isnull()==False, 'Year'].astype(str).astype(int)

In [94]:
df.loc[df['Year']==0, 'Year'] = None
df.loc[df['Year']>2021, 'Year'] = 2021
for brand in df['Make'].unique():
    df.loc[(df['Make']==brand) & (df['Year'].isnull()), 'Year'] = round(df.loc[(df['Make']==brand) & (df['Year'].isnull())==False, 'Year'].mean())
df['Year'].value_counts(dropna = False)

2018.0    6773
2017.0    4607
2020.0    3509
2019.0    3507
2021.0    3131
2016.0    2164
2015.0    2025
2014.0    1826
2013.0    1473
2012.0     995
2011.0     876
2008.0     649
2007.0     619
2010.0     572
2006.0     558
2005.0     504
1969.0     468
1967.0     438
2009.0     419
2004.0     403
1966.0     363
2003.0     360
1970.0     359
2002.0     341
1968.0     338
1965.0     333
1972.0     285
1957.0     267
1971.0     264
NaN        257
1995.0     231
2001.0     230
1973.0     213
1964.0     207
1994.0     206
2000.0     206
1979.0     205
1955.0     200
1999.0     200
1996.0     189
1978.0     184
1963.0     184
1987.0     184
1986.0     175
1956.0     173
1974.0     168
1993.0     167
1991.0     161
1997.0     156
1988.0     156
1977.0     154
1992.0     150
1989.0     148
1976.0     145
1985.0     140
1962.0     138
1998.0     138
1980.0     136
1990.0     128
1932.0     116
1984.0     115
1960.0     107
1959.0     107
1954.0     105
1975.0     104
1961.0      95
1948.0    

In [95]:
df.loc[df['Drive Type'].isnull(), 'Drive Type'] = df.loc[df['Drive Type'].isnull(), 'Drivetrain']
df.loc[df['Drive Type'].str.contains('(?i)4', na = False), 'Drive Type'] = 'AWD'
df.loc[df['Drive Type'].str.contains('(?i)quat', na = False), 'Drive Type'] = 'AWD'
df.loc[df['Drive Type'].str.contains('(?i)all', na = False), 'Drive Type'] = 'AWD'
df.loc[df['Drive Type'].str.contains('(?i)awd', na = False), 'Drive Type'] = 'AWD'
for brand in df['Make'].unique():
    for model in df.loc[df['Make']==brand, 'Model'].unique():
        try:
            df.loc[(df['Make']==brand) & (df['Model']==model) & (df['Drive Type'].isnull()), 'Drive Type'] = df.loc[(df['Make']==brand) & (df['Model']==model), 'Drive Type'].value_counts()[0:1].index.tolist()
        except:
            pass

df.loc[df['Drive Type']!= 'AWD', 'Drive Type'] = '2WD'
df['Drive Type'].value_counts(dropna = False)

2WD    26907
AWD    19250
Name: Drive Type, dtype: int64

In [96]:
df.loc[df['Fuel Type'].isnull(), 'Fuel Type'] = df.loc[df['Fuel Type'].isnull(), 'Fuel']
df.loc[df['Fuel Type'].str.contains('(?i)elec',na = False ), 'Cylinders'] = 0
df.loc[df['Fuel Type'].str.contains('(?i)elec',na = False ), 'Engine'] = 0
df.loc[df['Fuel Type'].str.contains('(?i)gaso',na = False ), 'Fuel Type'] = 'Gasoline'
df.loc[df['Fuel Type'].str.contains('(?i)die', na = False), 'Fuel Type'] = 'Diesel'
for brand in df['Make'].unique():
    for model in df.loc[df['Make']==brand, 'Model'].unique():
        try:
            df.loc[(df['Make']==brand) & (df['Model']==model) & (df['Fuel Type'].isnull()), 'Fuel Type'] = df.loc[(df['Make']==brand) & (df['Model']==model), 'Fuel Type'].value_counts()[0:1].index.tolist()
        except:
            pass
df.loc[(df['Fuel Type']!='Gasoline') & (df['Fuel Type']!='Diesel'), 'Fuel Type'] = 'Other'
df['Fuel Type'].value_counts(dropna = False)

Gasoline    40804
Other        3020
Diesel       2333
Name: Fuel Type, dtype: int64

In [97]:
df.loc[df['Condition']=='Certified pre-owned', 'Condition'] = 'Used'
df.loc[(df['Condition']=='None') & (df['Year']<2021), 'Condition'] = 'Used'
df.loc[(df['Condition']=='None') & (df['Year']==2021), 'Condition'] = 'New'
df.loc[(df['Condition'].isnull()) & (df['Year']<2021), 'Condition'] = 'Used'
df.loc[(df['Condition'].isnull()) & (df['Year']==2021), 'Condition'] = 'New'

df['Condition'].value_counts(dropna = False)

Used    42692
New      3400
NaN        65
Name: Condition, dtype: int64

In [98]:
for vol in df['Engine'].unique():
    try:
        df.loc[(df['Engine']==vol) & (df['Cylinders'].isnull()), 'Cylinders'] = df.loc[df['Engine']==vol, 'Cylinders'].value_counts()[0:1].index.tolist()
    except:
        pass
    
df.loc[df['Cylinders'].isnull() == False, 'Cylinders'] = df.loc[df['Cylinders'].isnull()==False, 'Cylinders'].astype(str).astype(int)

for brand in df['Make'].unique():
    for model in df.loc[df['Make']==brand, 'Model'].unique():
        try:
            df.loc[(df['Make']==brand) & (df['Model']==model) & (df['Cylinders'].isnull()), 'Cylinders'] = df.loc[(df['Make']==brand) & (df['Model']==model), 'Cylinders'].value_counts()[0:1].index.tolist()
        except:
            pass
        
for brand in df['Make'].unique():
    for body in df.loc[df['Make']==brand, 'Body Type'].unique():
        try:
            df.loc[(df['Make']==brand) & (df['Body Type']==body) & (df['Cylinders'].isnull()), 'Cylinders'] = df.loc[(df['Make']==brand) & (df['Body Type']==body), 'Cylinders'].value_counts()[0:1].index.tolist()
        except:
            pass

df.loc[df['Cylinders'].isnull(), 'Cylinders'] = round(df['Cylinders'].mean()).astype(int)
        
df['Cylinders'].value_counts(dropna = False)

8     16414
4     14014
6     13255
0       926
12      680
10      313
3       257
5       149
7       104
2        40
1         5
Name: Cylinders, dtype: int64

In [99]:
for cyl in df['Cylinders'].unique():
    try:
        df.loc[(df['Cylinders']==cyl) & (df['Engine'].isnull()), 'Engine'] = df.loc[df['Cylinders']==cyl, 'Engine'].value_counts()[0:1].index.tolist()
    except:
        pass

for brand in df['Make'].unique():
    for model in df.loc[df['Make']==brand, 'Model'].unique():
        try:
            df.loc[(df['Make']==brand) & (df['Model']==model) & (df['Engine'].isnull()), 'Engine'] = df.loc[(df['Make']==brand) & (df['Model']==model), 'Engine'].value_counts()[0:1].index.tolist()
        except:
            pass

for brand in df['Make'].unique():
    for body in df.loc[df['Make']==brand, 'Body Type'].unique():
        try:
            df.loc[(df['Make']==brand) & (df['Body Type']==body) & (df['Engine'].isnull()), 'Engine'] = round(df.loc[(df['Make']==brand) & (df['Body Type']==body), 'Engine'].mean(),1)
        except:
            pass

df.loc[df['Engine'].isnull(), 'Engine'] = round(df['Engine'].mean(),1)

df['Engine'].value_counts(dropna = False)

2.0    6672
3.0    5490
3.5    2746
3.6    2308
4.0    2153
1.2    1764
2.5    1630
1.0    1488
2.3    1281
5.0    1260
1.4    1210
1.5    1193
5.7    1164
6.7    1038
1.1     958
2.4     943
6.0     916
3.7     809
2.9     805
3.3     696
2.7     681
0.0     680
4.6     650
1.6     644
1.3     633
2.1     523
1.8     483
3.4     462
3.8     436
3.2     415
2.2     386
2.6     373
4.7     367
2.8     365
3.9     279
3.1     278
4.8     220
4.5     198
1.9     177
4.4     152
4.1     142
5.8     136
5.9     117
1.7     112
4.2     104
5.6      91
4.9      82
6.8      79
7.0      59
4.3      58
5.2      57
5.1      38
6.2      22
5.3      21
5.4      18
8.0      16
5.5      15
6.6      15
0.9      11
6.4       4
6.5       4
7.3       4
6.9       4
6.1       3
0.3       3
9.8       3
8.6       3
7.8       2
7.4       2
8.1       2
6.3       1
8.2       1
8.5       1
9.0       1
Name: Engine, dtype: int64

In [100]:
#df = df.dropna()
df.shape

(46157, 25)

In [101]:
df.columns

Index(['Unnamed: 0', 'Body Type', 'Condition', 'Cylinders', 'Engine',
       'Exterior Color', 'For Sale By', 'Interior Color', 'Make', 'Mileage',
       'Model', 'Price', 'Transmission', 'Vehicle Title', 'Warranty', 'Year',
       'Number of Cylinders', 'Drive Type', 'Fuel Type', 'Drivetrain', 'Fuel',
       'Engine Description', 'Turbo', 'USD', 'CAD'],
      dtype='object')

In [102]:
df = df.drop(['Unnamed: 0', 'Number of Cylinders', 'Drivetrain', 'Fuel', 'Engine Description', 'USD', 'CAD'], axis=1)
df.columns

Index(['Body Type', 'Condition', 'Cylinders', 'Engine', 'Exterior Color',
       'For Sale By', 'Interior Color', 'Make', 'Mileage', 'Model', 'Price',
       'Transmission', 'Vehicle Title', 'Warranty', 'Year', 'Drive Type',
       'Fuel Type', 'Turbo'],
      dtype='object')

In [103]:
df.head()

,Body Type,Condition,Cylinders,Engine,Exterior Color,For Sale By,Interior Color,Make,Mileage,Model,Price,Transmission,Vehicle Title,Warranty,Year,Drive Type,Fuel Type,Turbo
0,Other,Used,8,4.1,Other,Dealer,Tan,AMC,76882,Rebel,19350,Automatic,Clean,No,1970,2WD,Other,No
1,Convertible,Used,8,2.9,Other,Dealer,Black,AMC,118902,Other,10050,Automatic,Clean,No,1967,2WD,Gasoline,No
2,Coupe,Used,8,3,Gray,Private Owner,Other,AMC,53711,Javelin AMX 5.9,7500,Automatic,Clean,No,1974,2WD,Gasoline,No
3,Other,Used,8,2.7,Other,Dealer,Other,AMC,100,Javelin,45000,Automatic,Other,No,1970,2WD,Gasoline,No
4,Coupe,Used,6,2.9,Other,Dealer,Other,AMC,87000,Other,9800,Automatic,Clean,No,1965,2WD,Gasoline,No


In [104]:
df.to_csv('cars.csv')